# Approximate Killing Vector Fields on Meshes

This is a basic `python` implementation of *Approximate Killing vector fields* (AKVFs) to produce near-isometric deformations on meshes of 2D surfaces embedded in 3D.
For more details refer to
- [As-Killing-As-Possible Vector Fields for Planar Deformation](https://people.csail.mit.edu/jsolomon/assets/kvf_deformation.pdf) which introduce AKFVs for planar deformation (2D meshes in 2D space). Section (2.2) and Equations (5)-(7) describe how to set up the sparse least-squares system, which we generalize to 3D embedding space. Note, that we do not use the two-level optimization, because our deformations are prescribed only on the boundaries. Similarly, we do not use the logarithmic spiral trajectories, since we assume only small deformations.
- [Near-Isometric Level Set Tracking](https://people.csail.mit.edu/jsolomon/assets/akvftracking_compressed.pdf) discusses AKVFs on 2D surfaces in 3D, focusing on tracking instead of deformations.

The Killing energy of a domain $\Omega$ deformed along the deformation field $U$ is
$$ E_K (U) = \int_{p \in \Omega} || J_U(p) +J_U(p)^\top ||^2 $$
where we use the Frobenius norm and $\Omega$ is a 2D manifold in 3D.
Simimlarly, given a set of user-prescribed deformation handles $u_i$ at locations $C=\{ p_i \}$ the constraint violation is
$$ E_C (U) = \sum_{p_i \in C} ||U(p_i) - u_i||^2 $$

The as-Killing-as-possible deformation field $U$ is found by minimizing the Killing energy and constraint violation weighted by some scalar $\lambda$:
$$ U = \argmin_{\tilde{U}} \left( E_K(\tilde{U}) + \lambda E_C(\tilde{U}) \right) $$

On a mesh, we discretize the quantities and operators. For $n$ vertices and $m$ faces:
- The velocity $U \in \R^{n \times 3}$ is a vector on each vertex.
- The gradient $G \in \R^{3m \times n}$ discretizes $\nabla f = G f$ (mapping a scalar $f$ on each vertex to a vector $\nabla f$ of each face).

We need to take care of the shapes and the order of entries in these and energy matrices. Please note the comments in the code.
Furthermore, the mesh quality plays a role. Using meshes extracted with marching-cubes did not work well because the solver could not converge.


The implementation relies on `igl` python-bindings to compute the gradients, which internally use `scipy.sparse`, which we will also use to solve the sparse least-squares system.
For plotting, we use `k3d` and additionally `meshplot` to plot face attributes.

In [66]:
import igl
import scipy as sp
import numpy as np
import k3d

In [2]:
## Import mesh
v, f = igl.read_triangle_mesh("data/bunny.obj")
n = len(v)
m = len(f)

### Define constraints

In [56]:
## Ear: bend
pt_ear = np.array([-.02, .17, -.015])
r_ear = 0.03
idxs_ear = np.where(np.square(v-pt_ear).sum(1) < np.square(r_ear))[0]
dv_ear = np.zeros([len(idxs_ear), 3]) + [.01,0,.01]

## Base: anchor
idxs_base = np.where(v[:,1] < (v.min(axis=0)[1] + .002))[0]
dv_base = np.zeros([len(idxs_base), 3])


## Combine
idxs = np.hstack([
    idxs_ear,
    idxs_base,  ## comment this, if you want to demonstrate how rigid deformation is recovered
])

## The constraints dv are shape (|C|, 3)
dv = np.zeros(v.shape)
dv[idxs_ear] = dv_ear
dv[idxs_base] = dv_base ## comment this, if you want to demonstrate how rigid deformation is recovered

In [104]:
fig = k3d.plot() 
fig += k3d.mesh(v.astype(np.float32), f.astype(np.uint32), color=0xaaaaaa, opacity=.5)
fig += k3d.points(v[idxs_base], color=0xff0000, point_size=0.001)
fig += k3d.vectors(v[idxs_ear], dv[idxs_ear], color=0xff0000, line_width=0.0001, head_size=0.01)
fig.display()

Output()

In [105]:
fig.camera = [0, .2, .15, 0, .1, 0, 0, .95, -.25]

### Least-squares

In [4]:
## Discrete gradient operator
G = igl.grad(v, f)  ## shape (3*|F|, |V|)
## Split into gradients wrt x,y,z 
Gx, Gy, Gz = G[:m], G[m:2*m], G[2*m:]

## P is the discrete operator taking vertices to Killing energy in Forbenius norm
## Each row corresponds to a different term when expanding ||J+J'||^2

S = np.sqrt(2)
P = sp.sparse.bmat([
    [2*Gx, None, None],    ## (2*du_x/dx)^2
    [None, 2*Gy, None],    ## (2*du_y/dy)^2
    [None, None, 2*Gz],    ## (2*du_y/dz)^2
    [S*Gy, S*Gx, None],    ## 2*(du_x/dy + du_y/dx)^2
    [S*Gz, None, S*Gx],    ## 2*(du_y/dz + du_z/dy)^2
    [None, S*Gz, S*Gy],    ## 2*(du_x/dx + du_z/dx)^2
])

## Due to how this block matrix is created, every column represents a component of du.
## That means, that the unknown du is a stacked vector [du_x, du_y, du_z].
## This corresponds to flattening and reshaping with order='F' as opposed to 'C'.

In [75]:
## Minimize via lstsq ##

## First build the matrix for the constraint energy
k = len(idxs)
I = sp.sparse.csr_matrix((k,n), dtype=np.float64)
for i, idx in enumerate(idxs):
    I[i, idx] = 1
Ik = sp.sparse.bmat([
    [1*I,0*I,0*I],
    [0*I,1*I,0*I],
    [0*I,0*I,1*I]
])

## Run lstsq
lmbd = 1e2
A = sp.sparse.bmat([[P], [lmbd*Ik]])
b = np.hstack([np.zeros([P.shape[0]]), lmbd*dv[idxs].flatten(order='F')])

## Extract the solution
sol = sp.sparse.linalg.lsqr(A, b)
du = sol[0].reshape(v.shape, order='F')

### Plot

In [108]:
## Plot the solution

fig = k3d.plot() 
fig += k3d.mesh(v, f, color=0xaaaaaa, opacity=.5) ## Initial mesh
fig += k3d.mesh(v+du, f, attribute=np.sqrt(np.square(du).sum(1))) ## Deformed mesh with color-coded magnitude of deformation
fig.display()

C:\Users\artursb\Anaconda3\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "int32" does not match required type "uint32". A coerced copy has been created.
  warnings.warn(


Output()

In [109]:
fig.camera = [0, .2, .15, 0, .1, 0, 0, .95, -.25]

The resulting deformation seems reasonable. We can also demonstrate, that this recovers rigid deformations, if we remove the anchors at the base of the shape by commenting them out during the constraint definition. The resulting deformation is a pure translation. To see it better, you might want to tune the $\lambda$.

Lastly, we can visualize the Killing-energy on the shape.

In [65]:
## We can plot the Killing-energy ##

## Find the Killing energy on each face
e_K = np.square(P@sol[0]).reshape(len(f),-1, order='F').sum(-1)

## Plot w ith meshplot, becase k3d cannot plot face attributes
import meshplot as mp
mp.plot(v+du, f, e_K)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014100…